# BackTesting 


In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,
2015-04-04,253.697006,266.058734,255.661524,258.734938,307.338692,47.508324,-4.193133,-3.906538,260.597992,1
2015-04-05,260.597992,265.654567,256.131663,258.807999,306.873610,52.432292,-3.252122,-3.775655,255.492004,0
2015-04-06,255.492004,264.962268,256.070744,258.677960,306.362350,48.785829,-2.885117,-3.597547,253.179993,0
2015-04-07,253.179993,264.256468,255.795434,258.462354,305.833173,47.185618,-2.749132,-3.427864,245.022003,0
2015-04-08,245.022003,262.770301,254.769393,257.935281,305.228086,41.956153,-3.262042,-3.394700,243.675995,0
...,...,...,...,...,...,...,...,...,...,...
2024-12-12,100043.000000,95711.704687,96622.993102,88474.185367,71606.585559,60.775607,3382.945703,3915.653792,101459.257812,1
2024-12-13,101459.257812,96074.207812,97083.589741,88983.403894,71903.627074,62.898526,3378.370898,3808.197213,101372.968750,0
2024-12-14,101372.968750,96544.959115,97492.102028,89469.269183,72196.854355,62.675951,3329.403238,3712.438418,104298.695312,1
